In [1]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import mapping

import ipyvuetify as v
from utils import messages as ms
from utils import parameters as pm

In [2]:
#input and output as mutable objects 
class Table_io:
    def __init__(self):
        #input 
        self.file = None
        self.lng = None
        self.lat = None
        self.id = None
        
        #output
        self.pts = None
        
tb_io = Table_io()

In [3]:
#output to display messages
file_output = wf.OutputWidget(ms.TABLE_INTRO)

#btn 
file_btn = wf.ProcessBtn(ms.TABLE_BTN)

#create the inputs
file_select = v.Select(items=pm.getTxt(), label=ms.FILE_SELECT, v_model=None)

#bind the inputs 
wb.bind(file_select, tb_io, 'file', file_output)

#create a file selector 
id_ = 'file_widget'
title = "Select File"

file_tile = wf.Tile(
    id_,
    title,
    btn=file_btn, 
    inputs=[file_select],
    output=file_output
)


In [4]:
#tile to select columns 

col_output = wf.OutputWidget(ms.COL_INTRO)

#btn 
col_btn = wf.ProcessBtn(ms.COL_BTN)

#create the inputs
col_lng = v.Select(items=[], label=ms.COL_LNG, v_model=None)
col_lat = v.Select(items=[], label=ms.COL_LAT, v_model=None)
col_id = v.Select(items=[], label=ms.COL_ID, v_model=None)

#bind the inputs 
wb.bind(col_lng, tb_io, 'lng', col_output)
wb.bind(col_lat, tb_io, 'lat', col_output)
wb.bind(col_id, tb_io, 'id', col_output)

#create a file selector 
id_ = 'file_widget'
title = "Select Columns"

col_tile = wf.Tile(
    id_,
    title,
    btn=col_btn, 
    inputs=[col_lng, col_lat, col_id],
    output=col_output
)

In [5]:
#tiles to display the points on a map
tb_map = m = mapping.init_map()[1]
title = 'Display points'
tb_map_tile = wf.Tile(
    id_,
    title,
    inputs=[tb_map]
)

In [6]:
from scripts import table as run
from sepal_ui.scripts import utils as su
from functools import partial

In [7]:
#function to actualise the values of the select column tile 
def update_col(cols, values):
    for col in cols:
        col.items = values

In [8]:
import pandas as pd

#validate the points selection 
def process_start(widget, event, data, output, cols):
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    #load the inputs 
    file = getattr(tb_io, 'file')
    
    #check input
    if not wb.checkInput(file, output, ms.NOT_A_FILE): return su.toggleLoading(widget)
    
    #check the file
    error = run.isConform(file)
    if error:
        su.displayIO(output, error, 'error')
        return su.toggleLoading(widget) 
    
    #read the columns names
    df = pd.read_csv(file)
    update_col(cols, df.columns.tolist())
    
    #write the values in the output
    su.displayIO(output, ms.READ_COLUMNS.format(', '.join(df.columns)), 'success')
    
    
    #display the map
    #pts = run.setMap(file, m)  
    
    #set the output value
    #setattr(tb_io, 'pts', pts)
    
    #toggle the loading button 
    su.toggleLoading(widget)
    
    return 

file_btn.on_event('click', partial(
    process_start,
    output = file_output,
    cols = [col_lng, col_lat, col_id]
))

In [9]:
file_tile

Layout(align_center=True, children=[Card(children=[Html(children=['Select File'], tag='h2'), Flex(children=[La…

TraitError: The 'items' trait of a Select instance must be a list or None, but a value of class 'pandas.core.indexes.base.Index' (i.e. Index(['LON', 'LAT', 'PLOTID'], dtype='object')) was specified.

In [10]:
col_tile

Layout(align_center=True, children=[Card(children=[Html(children=['Select Columns'], tag='h2'), Flex(children=…

In [11]:
tb_map_tile

Layout(align_center=True, children=[Card(children=[Html(children=['Display points'], tag='h2'), Flex(children=…

In [12]:
import pandas as pd 

df = pd.read_csv('/home/prambaud/sample_csvPilot.csv')
df.columns

FileNotFoundError: [Errno 2] No such file or directory: '/home/prambaud/sample_csvPilot.csv'

In [ ]:
'manger [{}]'.format(', '.join(df.columns))